In [8]:
!pip install fastapi Pillow python-multipart torch transformers uvicorn nest-asyncio pyngrok

In [9]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")



In [10]:
# prepare image + question
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

text = "What are the colors of the cats?"

# prepare inputs
encoding = processor(image, text, return_tensors="pt")


In [11]:

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()


print("Predicted answer:", model.config.id2label[idx])

# TODO: put above code into a function that accepts image and text as input

Predicted answer: brown and black


In [12]:
def model_pipeline(text: str, image: Image):
    # prepare inputs
    encoding = processor(image, text, return_tensors="pt")

    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()

    return  model.config.id2label[idx]

In [ ]:
#used gemini to make this work
from fastapi import FastAPI, UploadFile, Form, File
from typing import Union
import io
from PIL import Image

# ... (rest of your imports and setup)
app = FastAPI()

@app.post("/ask")
# CRUCIAL FIX: Use Form(...) to tell FastAPI that 'text' is in the form-data body.
# Using File(...) for 'image' is also good practice for clarity.
def ask(text: str = Form(...), image: UploadFile = File(...)):
    
    # Also ensure you use the corrected file reading logic to avoid the 500 error:
    content = image.file.read()
    
    # Check for empty content (good practice)
    if not content:
        # You could return a 400 error here if you want a clearer client error
        # return JSONResponse(status_code=400, content={"detail": "Uploaded image file is empty."})
        pass
        
    pil_image = Image.open(io.BytesIO(content))

    result = model_pipeline(text, pil_image)
    
    return {"answer": result}






In [15]:
import nest_asyncio
nest_asyncio.apply()
from pyngrok import ngrok
import uvicorn

In [ ]:



ngrok.set_auth_token("token")

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
config = uvicorn.Config(app, port=8000, log_level="info")
server = uvicorn.Server(config)

import asyncio
# Use asyncio to serve, compatible with nest_asyncio
await server.serve()

